In [1]:
%pip install -q langchain==0.2.8 langchain-chroma==0.1.2 chromadb==0.4.18

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install -q langchain==0.2.8
%pip install transformers torch einops
%pip install 'numpy<2'

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ЌҐ г¤ Ґвбп ­ ©вЁ гЄ § ­­л© д ©«.


In [3]:
import requests
import numpy as np
# import faiss
import os

In [21]:
texts = [os.path.join('./lectures', text) for text in os.listdir('./lectures')]
g = open('essay.txt', 'w')
for text in texts:
    with open(text, "r",  encoding="utf-8") as f:
        t = f.read()
        g.write(t)
g.close()

In [4]:
with open('essay.txt', "r") as g:
    text = g.read()
    chunk_size = 2048
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [5]:
len(chunks)

37

In [8]:
from transformers import AutoModel

# Initialize the model
model = AutoModel.from_pretrained("jinaai/jina-embeddings-v2-small-en", trust_remote_code=True)

# When calling the `encode` function, you can choose a `task` based on the use case:
# 'retrieval.query', 'retrieval.passage', 'separation', 'classification', 'text-matching'
# Alternatively, you can choose not to pass a `task`, and no specific LoRA adapter will be used.
embeddings = model.encode(chunks)

print(embeddings[0])

[-9.13396031e-02 -1.75151527e-01 -1.59966405e-02  2.91613042e-01
 -2.54094917e-02 -2.89567113e-01  8.36245045e-02  5.86864352e-02
 -1.53837567e-02  4.16542776e-02  1.47793859e-01 -3.97765823e-02
 -2.23270729e-01 -1.45194709e-01  1.42224446e-01 -3.12424630e-01
 -1.01314634e-02 -2.82872856e-01 -1.13619506e-01 -2.04220891e-01
 -3.52307670e-02 -3.44558716e-01  1.28138453e-01 -3.33247989e-01
  3.71965677e-01 -1.15847617e-01 -3.29494148e-01 -4.28082794e-02
  2.37138689e-01  2.59559572e-01 -8.03757180e-03  9.60875005e-02
 -2.56749708e-02  2.69982934e-01 -3.97022367e-01  1.31641686e-01
  2.65833765e-01  5.26349247e-02 -1.18016802e-01 -3.69086742e-01
 -2.69569248e-01  1.44196795e-02  2.81785488e-01  1.20239407e-01
 -3.71896654e-01  8.03141072e-02 -4.20731455e-01  2.33962432e-01
  4.49521318e-02 -1.84322000e-02 -3.01245362e-01  2.18079373e-01
 -6.29352808e-01  5.02580479e-02 -3.08231153e-02  2.89443612e-01
 -7.41175339e-02  1.80217445e-01 -2.20154822e-01 -9.57682878e-02
  1.98798239e-01  1.06709

In [24]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(api_key='',model_name="")
faiss_index = FAISS.from_texts(chunks, embedding=embeddings)